In [1]:
require("tm")

Loading required package: tm
Loading required package: NLP


In [2]:
require("topicmodels")

Loading required package: topicmodels


In [3]:
load(url("https://github.com/RFJHaans/topicmodeling/blob/master/LDA200.RData?raw=true"))

In [4]:
dtm

ERROR: Error in eval(expr, envir, enclos): object 'dtm' not found


In [5]:
load(url("https://github.com/RFJHaans/topicmodeling/blob/master/LDA200.RData?raw=true"))

In [6]:
dtm

ERROR: Error in eval(expr, envir, enclos): object 'dtm' not found


In [7]:
data = read.csv(url("https://raw.githubusercontent.com/RFJHaans/topicmodeling/master/ASQ_AMJ_AMR_OS_SMJ.csv"))

In [8]:
corpus = VCorpus((VectorSource(data[, "AB"])))

In [9]:
corpusclean = tm_map(corpus, removeNumbers)
# 2) Remove punctuation
    corpusclean = tm_map(corpusclean, removePunctuation)
# 3) Transform all upper-case letters to lower-case.
    corpusclean = tm_map(corpusclean,  content_transformer(tolower))
# 4) Remove stopwords which do not convey any meaning.
    corpusclean = tm_map(corpusclean, removeWords, stopwords("english"))
# this stopword file is at C:\Users\[username]\Documents\R\win-library\[rversion]\tm\stopwords 

# i	me	my	myself	we	our	ours	ourselves	you	your	yours	yourself	yourselves	he	him	his	himself	
# she	her	hers	herself	it	its	itself	they	them	their	theirs	themselves	what	which	who	whom	this
# that	these	those	am	is	are	was	were	be	been	being	have	has	had	having	do	does	did	doing	would	should
# could	ought	i'm	you're	he's	she's	it's	we're	they're	i've	you've	we've	they've	i'd	you'd	he'd	she'd	we'd
# they'd	i'll	you'll	he'll	she'll	we'll	they'll	isn't	aren't	wasn't	weren't	hasn't	haven't	hadn't	doesn't	
# don't	didn't	won't	wouldn't	shan't	shouldn't	can't	cannot	couldn't	mustn't	let's	that's	who's	what's	here's
# there's	when's	where's	why's	how's	a	an	the	and	but	if	or	because	as	until	while	of	at	by	for	with	about	
# against	between	into	through	during	before	after	above	below	to	from	up	down	in	out	on	off	over	under	again
# further	then	once	here	there	when	where	why	how	all	any	both	each	few	more	most	other	some	such	no	nor	
# not	only	own	same	so	than	too	very

# 5) And strip whitespace. 
    corpusclean = tm_map(corpusclean , stripWhitespace)

# See the help of getTransformations for more possibilities, such as stemming. 

# To speed up the computation process for this tutorial, I have selected some choice words that were very common:
# We update the corpusclean corpus by removing these words. 
    corpusclean = tm_map(corpusclean, removeWords, c("also","based","can","data","effect",
                                                  "effects","elsevier","evidence","examine",
                                                  "find","findings","high","low","higher","lower",
                                                  "however","impact","implications","important",
                                                  "less","literature","may","model","one","paper",
                                                  "provide","research","all rights reserved",
                                                  "results","show","studies","study","two","use",
                                                  "using","rights","reserved","new","analysis","three",
                                                  "associated","firm","firms","copyright","sons","john","ltd","wiley"))

### Adding metadata from the original database
# This needs to be done because transforming things into a corpus only uses the texts.
    i = 0
    corpusclean = tm_map(corpusclean, function(x) {
      i <<- i +1
      meta(x, "id") = as.character(data[i,"ID"])
      x
    })

    i = 0
    corpusclean = tm_map(corpusclean, function(x) {
      i <<- i +1
      meta(x, "journal") = as.character(data[i,"SO"])
      x
    })

# The above is a loop that goes through all files ("i") in the corpus
# and then maps the information of the metadata dataframe 
# (the "ID" column, et cetera) to a new piece of metadata in the corpus
# which we also call "id", et cetera.

# This enables making selections of the corpus based on metadata now.
# Let's say we want to only look at articles from the AMJ, then we do the following:
    keep = meta(corpusclean, "journal") == "ACADEMY OF MANAGEMENT JOURNAL"
    corpus.AMJ = corpusclean[keep]

# We then convert the corpus to a "Document-term-matrix" (dtm)
    dtm =DocumentTermMatrix(corpusclean)  
    dtm

<<DocumentTermMatrix (documents: 1530, terms: 11744)>>
Non-/sparse entries: 89755/17878565
Sparsity           : 100%
Maximal term length: 30
Weighting          : term frequency (tf)